In [14]:
import yaml
from pystac import STAC_IO, read_file, Item, Catalog, CatalogType
from urllib.parse import urlparse
import requests
import os

## Stage-in remote STAC items as a local STAC Catalog

In [15]:
def my_read_method(uri):
    
    parsed = urlparse(uri)
    
    if parsed.scheme.startswith('http'):
        return requests.get(uri).text
    else:
        return STAC_IO.default_read_text_method(uri)

In [16]:
input_references = ['https://terradue-rtd.gitlab.io/sentinel-s2-l2a-cogs/53/H/PA/S2B_MSIL2A_20191012T004709_N0213_R102_T53HPA_20191012T023414/S2B_MSIL2A_20191012T004709_N0213_R102_T53HPA_20191012T023414.json', 
                    'https://terradue-rtd.gitlab.io/sentinel-s2-l2a-cogs/53/H/PA/S2B_MSIL2A_20200130T004659_N0213_R102_T53HPA_20200130T022348/S2B_MSIL2A_20200130T004659_N0213_R102_T53HPA_20200130T022348.json',
                    'https://terradue-rtd.gitlab.io/sentinel-s2-l2a-cogs/53/H/PA/S2B_MSIL2A_20200209T004659_N0214_R102_T53HPA_20200209T022720/S2B_MSIL2A_20200209T004659_N0214_R102_T53HPA_20200209T022720.json']

In [17]:
STAC_IO.read_text_method = my_read_method
    
catalogs = []

for index, input_reference in enumerate(input_references):

    items = []
    
    thing = read_file(input_reference)

    if isinstance(thing, Item):

        items.append(thing)

    elif isinstance(thing, Catalog):

        for item in thing.get_items():

            items.append(item)

    # create catalog
    catalog = Catalog(id=items[0].id,
              description='staged STAC catalog with {}'.format(items[0].id))

    catalog.add_items(items)

    catalog.normalize_and_save(root_href=items[0].id,
                               catalog_type=CatalogType.RELATIVE_PUBLISHED)

    catalog.describe()
    
    catalogs.append(os.path.dirname(catalog.get_self_href()))

2020-11-25T16:45:09 DEBUG    Starting new HTTPS connection (1): terradue-rtd.gitlab.io:443
2020-11-25T16:45:09 DEBUG    https://terradue-rtd.gitlab.io:443 "GET /sentinel-s2-l2a-cogs/53/H/PA/S2B_MSIL2A_20191012T004709_N0213_R102_T53HPA_20191012T023414/S2B_MSIL2A_20191012T004709_N0213_R102_T53HPA_20191012T023414.json HTTP/1.1" 200 17169
2020-11-25T16:45:09 DEBUG    Starting new HTTPS connection (1): terradue-rtd.gitlab.io:443
2020-11-25T16:45:10 DEBUG    https://terradue-rtd.gitlab.io:443 "GET /sentinel-s2-l2a-cogs/53/H/PA/S2B_MSIL2A_20200130T004659_N0213_R102_T53HPA_20200130T022348/S2B_MSIL2A_20200130T004659_N0213_R102_T53HPA_20200130T022348.json HTTP/1.1" 200 17119


* <Catalog id=S2B_MSIL2A_20191012T004709_N0213_R102_T53HPA_20191012T023414>
  * <Item id=S2B_MSIL2A_20191012T004709_N0213_R102_T53HPA_20191012T023414>


2020-11-25T16:45:10 DEBUG    Starting new HTTPS connection (1): terradue-rtd.gitlab.io:443
2020-11-25T16:45:10 DEBUG    https://terradue-rtd.gitlab.io:443 "GET /sentinel-s2-l2a-cogs/53/H/PA/S2B_MSIL2A_20200209T004659_N0214_R102_T53HPA_20200209T022720/S2B_MSIL2A_20200209T004659_N0214_R102_T53HPA_20200209T022720.json HTTP/1.1" 200 17116


* <Catalog id=S2B_MSIL2A_20200130T004659_N0213_R102_T53HPA_20200130T022348>
  * <Item id=S2B_MSIL2A_20200130T004659_N0213_R102_T53HPA_20200130T022348>
* <Catalog id=S2B_MSIL2A_20200209T004659_N0214_R102_T53HPA_20200209T022720>
  * <Item id=S2B_MSIL2A_20200209T004659_N0214_R102_T53HPA_20200209T022720>


In [18]:
catalogs

['/home/jovyan/test-workflows/expert-mode/S2B_MSIL2A_20191012T004709_N0213_R102_T53HPA_20191012T023414',
 '/home/jovyan/test-workflows/expert-mode/S2B_MSIL2A_20200130T004659_N0213_R102_T53HPA_20200130T022348',
 '/home/jovyan/test-workflows/expert-mode/S2B_MSIL2A_20200209T004659_N0214_R102_T53HPA_20200209T022720']

## Expert mode - multitemporal RGB composite

In [19]:
with open(r'scombi-do-expert-mt-raster.yml') as file:
    
    cwl_params = yaml.load(file, Loader=yaml.FullLoader)

In [23]:
cwl_params

{'aoi': 'POLYGON((136.707 -35.991,136.707 -35.804,137.071 -35.804,137.071 -35.991,136.707 -35.991))',
 'band_1': 'red',
 'band_2': 'green',
 'band_3': 'blue',
 'color': 'Gamma RGB 3.5 Saturation 1.4 Sigmoidal RGB 15 0.45',
 'input_reference_1': {'class': 'Directory',
  'path': '/home/jovyan/test-workflows/expert-mode/S2B_MSIL2A_20191012T004709_N0213_R102_T53HPA_20191012T023414'},
 'input_reference_2': {'class': 'Directory',
  'path': '/home/jovyan/test-workflows/expert-mode/S2B_MSIL2A_20200130T004659_N0213_R102_T53HPA_20200130T022348'},
 'input_reference_3': {'class': 'Directory',
  'path': '/home/jovyan/test-workflows/expert-mode/S2B_MSIL2A_20200209T004659_N0214_R102_T53HPA_20200209T022720'},
 'resolution': 'highest',
 's_expression': ['(interp v1 (asarray 0 10000) (asarray 0 1))',
  '(interp v2 (asarray 0 10000) (asarray 0 1))',
  '(interp v3 (asarray 0 10000) (asarray 0 1))']}

In [24]:
cwl_params['input_reference_1']['path'] = catalogs[0]
cwl_params['input_reference_2']['path'] = catalogs[1]
cwl_params['input_reference_3']['path'] = catalogs[2]

In [25]:
cwl_params

{'aoi': 'POLYGON((136.707 -35.991,136.707 -35.804,137.071 -35.804,137.071 -35.991,136.707 -35.991))',
 'band_1': 'red',
 'band_2': 'green',
 'band_3': 'blue',
 'color': 'Gamma RGB 3.5 Saturation 1.4 Sigmoidal RGB 15 0.45',
 'input_reference_1': {'class': 'Directory',
  'path': '/home/jovyan/test-workflows/expert-mode/S2B_MSIL2A_20191012T004709_N0213_R102_T53HPA_20191012T023414'},
 'input_reference_2': {'class': 'Directory',
  'path': '/home/jovyan/test-workflows/expert-mode/S2B_MSIL2A_20200130T004659_N0213_R102_T53HPA_20200130T022348'},
 'input_reference_3': {'class': 'Directory',
  'path': '/home/jovyan/test-workflows/expert-mode/S2B_MSIL2A_20200209T004659_N0214_R102_T53HPA_20200209T022720'},
 'resolution': 'highest',
 's_expression': ['(interp v1 (asarray 0 10000) (asarray 0 1))',
  '(interp v2 (asarray 0 10000) (asarray 0 1))',
  '(interp v3 (asarray 0 10000) (asarray 0 1))']}

In [26]:
with open(r'scombi-do-expert-mt-raster.yml', 'w') as file:
    params = yaml.dump(cwl_params, file)

Open a Terminal and run the CWL document with:

```console
cd /home/jovyan/test-workflows/expert-mode
cwltool --no-container scombi-do-expert.cwl#expert scombi-do-expert-mt-raster.yml 
```

Use the `demo/cwl-result-viewer.ipynb` Jupyter Notebook to view the results